In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import tensorflow_datasets as tfds
import math
import numpy as np
import matplotlib.pyplot as plt
import logging
from urllib import parse
from http.server import HTTPServer, BaseHTTPRequestHandler
logger = tf.get_logger()

logger.setLevel(logging.ERROR)

# Cargar datos de MNIST
dataset, metadata = tfds.load('mnist', as_supervised=True, with_info=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

# Normalizar imágenes (0-255 -> 0-1)
def normalize(images, labels):
    images = tf.cast(images, tf.float32)
    images /= 255
    return images, labels

train_dataset = train_dataset.map(normalize)
test_dataset = test_dataset.map(normalize)

# Convertir etiquetas a par (+1) o non (-1)
def binary_labels(image, label):
    return image, tf.where(label % 2 == 0, 1, -1)  # Par: +1, Non: -1

train_dataset = train_dataset.map(binary_labels)
test_dataset = test_dataset.map(binary_labels)

# Definir  (1 capa Dense con activación 'sign' personalizada)
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
    tf.keras.layers.Dense(1, activation='linear')  # Salida lineal para el Perceptrón
])

# Función de activación tipo "sign" (no incluida en Keras, se aplica después)
def sign_activation(z):
    return tf.where(z >= 0, 1, -1)

# Compilar el modelo con pérdida MSE (para Adaline)
model.compile(
    optimizer='sgd',  # Descenso de gradiente estándar
    loss='mean_squared_error',  # MSE para Adaline
    metrics=['accuracy']
)

# Entrenamiento (lotes de 32)
BATCHSIZE = 32
train_dataset = train_dataset.repeat().shuffle(metadata.splits['train'].num_examples).batch(BATCHSIZE)
test_dataset = test_dataset.batch(BATCHSIZE)

# Entrenamiento con historial
model.fit(
    train_dataset,
    epochs=15,   #numero de epocas
    steps_per_epoch=math.ceil(metadata.splits['train'].num_examples / BATCHSIZE),
    validation_data=test_dataset,
    validation_steps=math.ceil(metadata.splits['test'].num_examples / BATCHSIZE)
)


# Servidor HTTP para predicciones
class SimpleHTTPRequestHandler(BaseHTTPRequestHandler):
    def do_POST(self):
        content_length = int(self.headers['Content-Length'])
        content_length = int(self.headers['Content-Length'])
        data = self.rfile.read(content_length)
        data = data.decode().replace('pixeles=', '')
        data = parse.unquote(data)
        
         #Realizar transformacion para dejar igual que los ejemplos que usa MNIST
        arr = np.fromstring(data, np.float32, sep=",")
        arr = arr.reshape(28,28)
        arr = np.array(arr)
        arr = arr.reshape(1,28,28,1)
   
        
        # Predecir con Adaline (salida lineal continua)
        prediction = model.predict(arr, batch_size=1)
        # Aplicar activación "sign" solo para la clasificación final
        predicted_class = 1 if prediction[0][0] >= 0 else -1
        
        print(f"Predicción: {'Par' if predicted_class == 1 else 'Non'}")

        self.send_response(200)
        self.send_header("Access-Control-Allow-Origin", "*")
        self.end_headers()
        self.wfile.write(str(predicted_class).encode())

# Iniciar servidor
print("Iniciando el servidor...")
server = HTTPServer(('localhost', 8000), SimpleHTTPRequestHandler)
server.serve_forever()

Epoch 1/15


C:\Users\YaTuSabes\miniconda3\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.2617 - loss: 0.4808 - val_accuracy: 0.3076 - val_loss: 0.4028
Epoch 2/15
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.2870 - loss: 0.4115 - val_accuracy: 0.3231 - val_loss: 0.4087
Epoch 3/15
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.2881 - loss: 0.4052 - val_accuracy: 0.3003 - val_loss: 0.3993
Epoch 4/15
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.2939 - loss: 0.4004 - val_accuracy: 0.2656 - val_loss: 0.4016
Epoch 5/15
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.2859 - loss: 0.4028 - val_accuracy: 0.3103 - val_loss: 0.3961
Epoch 6/15
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.2921 - loss: 0.4014 - val_accuracy: 0.3088 - val_loss: 0.3991
Epoch 7/15
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.2915 - loss: 0.3962 - val_accuracy: 0.2463 - val_loss: 0.4012
Epoch 8/15
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.2980 - loss: 0.3993 - val_accurac

127.0.0.1 - - [27/Mar/2025 20:53:25] "POST / HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
Predicción: Par


...

127.0.0.1 - - [27/Mar/2025 20:58:00] "POST / HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Predicción: Par


127.0.0.1 - - [27/Mar/2025 20:58:05] "POST / HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
Predicción: Par


127.0.0.1 - - [27/Mar/2025 20:58:38] "POST / HTTP/1.1" 200 -
